<a href="https://colab.research.google.com/github/LiyaGaynutdinova/mlp_MNIST/blob/main/MLP_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Týden 11. Vícevrstvý perceptron v Pytorch.
Dneska ukážeme jak vytvářet, trénovat a používat neuronových sítí v PyTorchu. Sestavíme jednoduchou plně propojenou neuronovou síť, která klasifikuje ručně psané číslice na známé datové sadě MNIST.

## Instalace

Pokud používáte tento notebook na platformě Colab, nemusíte instalovat žádné knihovny. Pokud pracujete na lokálním počítači, máte doinstalot tyto knihovny:

* Pytorch
* Torchvision

Je doporučeno, nikoli však nutné, aby váš systém Windows byl vybaven grafickým procesorem NVIDIA, abyste mohli plně využít podporu CUDA v PyTorch. Umožňuje to mnohem rychlejší výpočty.

### Pip pro systémy s grafickým procesorem NVIDIA

pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

### Pip pro ostatní systémy

pip3 install torch torchvision torchaudio

### Conda pro systémy s grafickým procesorem NVIDIA

conda install pytorch torchvision torchaudio pytorch-cuda=11.7 -c pytorch -c nvidia

### Conda pro ostatní systémy

conda install pytorch torchvision torchaudio cpuonly -c pytorch

## Tensory

Než se pustíme do budování neuronových sítí, měli bychom si vysvětlit základní stavební kámen Pytorchovy knihovny – objekt `Tensor`. Torch Tensor je základní datová struktura knihovny PyTorch, určená pro vícerozměrná pole a matice a také pro uchovávání skalárních hodnot. Tenzory v PyTorch jsou optimalizovány pro úlohy hlubokého učení akcelerované GPU a obsahují rozsáhlou sadu funkcí a operací určených pro tyto úlohy. Jak uvidíte, inicializace tenzoru, manipulace a matematické operace vypadají podobně jako v Numpy, i když s některými důležitými rozdíly:

1. **Akcelerace GPU**: Tenzory Torch lze snadno přesunout na GPU, zatímco přesun pole NumPy na GPU vyžaduje další knihovny, jako je CuPy.
  
2. **Automatická diferenciace**: Tenzory PyTorch jsou vybaveny vestavěnou funkcí automatické diferenciace prostřednictvím balíčku `autograd`, která je nezbytná pro trénování neuronových sítí. Pole NumPy tuto funkci nativně nepodporují.

3. **Ekosystém knihoven**: Torch Tensors jsou přizpůsobeny pro úlohy hlubokého učení a bezproblémově se integrují s moduly neuronových sítí PyTorch. Pole NumPy jsou vhodnější pro úlohy, které nezahrnují neuronové sítě, a jsou součástí širšího ekosystému vědeckých výpočtů.

4. **Správa paměti**: Tensory Torch jsou optimalizovány pro výkon při rozsáhlých výpočtech a mohou v takových scénářích efektivněji nakládat s pamětí. Pole NumPy jsou obecně přímočařejší a může být snazší s nimi pracovat při výpočtech malého rozsahu.

5. **Interoperabilita**: PyTorch poskytuje nástroje pro převod mezi Torch Tensors a NumPy poli, ale jejich společné použití v rámci jednoho projektu může vyžadovat pečlivé zacházení, aby byla zajištěna kompatibilita.

Nyní můžeme importovat potřebné knihovny:

In [ ]:
import numpy as np
import torch

### Inicializace

Tenzory lze inicializovat různými způsoby:

Ze seznamu:

In [ ]:
a = torch.tensor([1, 2, 3])
print(a)

Nuly a jedničky:

In [ ]:
b = torch.zeros(3, 3)
c = torch.ones(2, 2)
print(b, c)

Náhodné hodnoty:

In [ ]:
d = torch.rand(2, 2)
print(d)

### Základní operace

Sčítání:

In [ ]:
e = a + a
print(e)

Násobení:

In [ ]:
f = a * 3
print(f)

### Přetváření
Tenzory můžete přetvářet pomocí metody `.view()`:

In [ ]:
g = torch.rand(4, 4)
h = g.view(16)
i = g.view(-1, 8)  # The size -1 is inferred from other dimensions
print(g, h, i)

### Automatická diferenciace

Vytvoření tenzoru se sledováním gradientu:

In [ ]:
x = torch.ones(3, 3, requires_grad=True)

Provádění operací:

In [ ]:
y = x + 5
z = y * y * 2
out = z.mean()

Výpočet gradientů:

In [ ]:
out.backward()

Zkontrolujme gradienty:

In [ ]:
print(x.grad)

### Jednoduchý příklad
Definujme jednoduchou funkci a najděme její derivaci pomocí `autograd`:

In [ ]:
# Define a tensor
x = torch.tensor(2.0, requires_grad=True)

# Define a function f(x) = x^2
y = x ** 2

# Compute the gradient
y.backward()

# Print the gradient
print(x.grad)  # Should print tensor(4.0)


## Neuronové sítě

Nejdříve naimportujeme dálší knihovny:

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

Pokud je váš počítač vybaven grafickým procesorem, můžete maticové operace počítat mnohem rychleji. Akceleraci GPU můžete v Colabu povolit výběrem možnosti Runtime -> Change runtime type -> Výběrem možnosti "GPU" z rozbalovací nabídky.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device} device")

Nyní můžeme vybudovat novou síť. Za tímto účelem vytvoříme novou instanci třídy nn.Module z knihovny torch.

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        y = self.fc3(x)
        return torch.softmax(y, dim=1), x

net = Net().to(device)
print(net)

Nyní je třeba připravit naši datovou sadu. Černobílé obrázky datové sady MNIST lze stáhnout pomocí knihovny torch. Poté je musíme převést na tenzor - speciální pole, které může uchovávat závislosti na jiných tenzorech, takže gradienty lze vypočítat automaticky.

In [ ]:
batch_size = 64

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST('data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

Můžeme se podívat na obrázky z datové sady:

In [ ]:
plt.imshow(train_dataset.__getitem__(1)[0].squeeze(), cmap='gray_r')

Nyní musíme definovat účelovou funkci: zde chceme minimalizovat počet nesprávně označených číslic. O zbytek se pak postará optimalizátor.

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

Při trénování musíme po každé epoše zkontrolovat chybu v testovací množině dat, abychom se ujistili, že model dobře zobecňuje.

In [ ]:
epochs = 10

train_losses = []
test_losses = []

for epoch in range(epochs):
    net.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs, _ = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        if i % 100 == 99:
            print(f"Epoch: {epoch + 1}, Batch: {i + 1}, Training Loss: {running_loss / 100:.3f}, Training Accuracy: {100 * correct / total:.2f}%")
            train_losses.append(running_loss / 100)
            running_loss = 0.0
            correct = 0
            total = 0

    net.eval()
    correct = 0
    total = 0
    test_loss = 0.0

    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            outputs, _ = net(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        test_loss /= len(test_loader)
        test_losses.append(test_loss)
        print(f"Epoch: {epoch + 1}, Test Loss: {test_loss:.3f}, Test Accuracy: {100 * correct / total:.2f}%")


Nyní můžeme vykreslit průběh tréninku.

In [ ]:
plt.plot(np.arange(1.1, 10.1, 0.1),train_losses, label='Training loss')
plt.plot(np.arange(1., 11.),test_losses, label='Test loss')
plt.legend()
plt.show()

Pravděpodobně vidíte, že dosažená přesnost je velmi vysoká, ale ne stoprocentní. Podívejme se na obrázky, které byly označeny nesprávně.

In [ ]:
net.eval()
correct = 0
total = 0
test_loss = 0.0

incorrect_images = []
incorrect_labels = []
incorrect_preds = []

with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        outputs, _ = net(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        for i in range(len(labels)):
            if predicted[i] != labels[i]:
                incorrect_images.append(inputs[i].cpu())
                incorrect_labels.append(labels[i].cpu())
                incorrect_preds.append(predicted[i].cpu())

test_loss /= len(test_loader)
print(f"Test Loss: {test_loss:.3f}, Test Accuracy: {100 * correct / total:.2f}%")

fig = plt.figure(figsize=(11, 11))

for i in range(25):
    ax = fig.add_subplot(5, 5, i + 1)
    ax.axis('off')
    ax.set_title(f"True: {incorrect_labels[i]}, Predicted: {incorrect_preds[i]}")
    ax.imshow(incorrect_images[i].squeeze(), cmap='gray_r')

plt.show()

Jak toho model dosahuje? Vidíme, že naše poslední skrytá vrstva je vektor o 64 rozměrech. Před přiřazením pravděpodobnosti, že číslice patří do určité třídy, sítě tento 64rozměrný prostor rozdělí, takže obrázky stejné číslice se seskupí. To se bude špatně vizualizovat, proto můžeme použít [t-distributed stochastic neighbor embedding](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding) k promítnutí těchto shluků do 2d prostoru.

In [ ]:
from sklearn.manifold import TSNE

net.eval()
hidden_activations = []
labels = []

with torch.no_grad():
    for data in test_loader:
        inputs, batch_labels = data
        inputs = inputs.to(device)

        # Get the activations of the last hidden layer
        _, h = net(inputs)
        hidden_activations.append(h.cpu().detach().numpy())
        labels.append(batch_labels)

hidden_activations = np.concatenate(hidden_activations)
labels = np.concatenate(labels)

# Use t-SNE to reduce the dimensionality of the activations to 2D
tsne = TSNE(n_components=2, perplexity=30, learning_rate=200, n_iter=1000, random_state=42)
tsne_embeddings = tsne.fit_transform(hidden_activations)

# Plot the t-SNE embeddings colored by their true labels
fig, ax = plt.subplots(figsize=(8, 8))
scatter = ax.scatter(tsne_embeddings[:, 0], tsne_embeddings[:, 1], c=labels, cmap='tab10')
legend = ax.legend(*scatter.legend_elements(), title="Labels")
ax.add_artist(legend)
plt.show()